In [27]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [28]:
seed = 7
np.random.seed(seed)

In [29]:
dataframe = pd.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

le = LabelEncoder()

X = dataset[:,0:60].astype(float)

Y = dataset[:,60]
Y=le.fit_transform(dataset[:,60])


from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, shuffle= True)

one_hot_train_labels=to_categorical(y_train)
one_hot_test_labels=to_categorical(y_test)

x_val=x_train[:25]
partial_x_train=x_train[25:]

y_val=one_hot_train_labels[:25]
partial_y_train=one_hot_train_labels[25:]

In [34]:
def create_baseline():
    model=Sequential()
    model.add(Dense(32,activation='relu',input_shape=(60,)))
    model.add(Dense(2,activation='softmax'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    
    model.fit(partial_x_train,partial_y_train,epochs=5,validation_data=(x_val,y_val))

In [35]:
create_baseline()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 114 samples, validate on 25 samples
Epoch 1/5
114/114 [==============================] - 0s 3ms/step - loss: 0.6880 - acc: 0.5263 - val_loss: 0.6536 - val_acc: 0.8800
Epoch 2/5
114/114 [==============================] - 0s 70us/step - loss: 0.6703 - acc: 0.5614 - val_loss: 0.6479 - val_acc: 0.8400
Epoch 3/5
114/114 [==============================] - 0s 70us/step - loss: 0.6603 - acc: 0.6140 - val_loss: 0.6397 - val_acc: 0.6800
Epoch 4/5
114/114 [==============================] - 0s 52us/step - loss: 0.6544 - acc: 0.6140 - val_loss: 0.6365 - val_acc: 0.8400
Epoch 5/5
114/114 [==============================] - 0s 52us/step - loss: 0.6500 - acc: 0.6140 - val_loss: 0.6340 - val_acc: 0.8800
